In [68]:
# Importar el conjunto de datos
from sklearn.datasets import load_iris

iris = load_iris()

X = iris.data
y = iris.target

print(X, y)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(120, 4) (30, 4) (120,) (30,)


In [70]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

resultados = knn.predict(X_test)

print(accuracy_score(y_test, resultados))

0.9666666666666667


In [71]:
from sklearn.naive_bayes import GaussianNB

bayes = GaussianNB()

bayes.fit(X_train, y_train)

resultados = bayes.predict(X_test)

print(accuracy_score(y_test, resultados))

0.9333333333333333


In [72]:
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

tipo_entrada = [('float_input', FloatTensorType([None, 4]))]

# Convertir el modelo
onnx_knn = convert_sklearn(knn, initial_types=tipo_entrada)
onnx_gnb = convert_sklearn(bayes, initial_types=tipo_entrada)

with open("knn_iris.onnx", "wb") as f:
    f.write(onnx_knn.SerializeToString())

with open("gnb_iris.onnx", "wb") as f:
    f.write(onnx_gnb.SerializeToString())
    

In [77]:
import onnxruntime as rt
import numpy as np

# Cargar el modelo
knn_onnx = rt.InferenceSession("knn_iris.onnx")

# Predecir
entrada = X_test.astype(np.float32)

#
nombre_entrada = knn_onnx.get_inputs()[0].name
nombre_salida = knn_onnx.get_outputs()[0].name

salida = knn_onnx.run([nombre_salida], {nombre_entrada: entrada})[0]

print(salida)

exactitud = accuracy_score(y_test, salida)
print(exactitud)

[0 2 1 2 1 1 2 2 2 1 1 0 0 0 0 0 1 0 1 2 1 1 2 0 1 1 2 0 0 0]
0.9666666666666667
